In [9]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'E:/Dissertation Data'
#datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [10]:
import pickle
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

#from glove import Corpus, Glove

from gensim.models import Word2Vec

In [11]:
from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

We then automatically score the million most strongly associated
n-grams (i.e., roughly the top 5% of the Poisson-ranked
list) for compositionality.

Using word2vec (Mikolov et al., 2013) with the parameters
found to be most effective by Baroni et al. (2014), we
build a word embedding vector for every simplex word in
the vocabulary (ca. 1M types), as well as for each MWE candidate.

* Continuous bag of words model with 400-dimensional vectors, window size 5, subsampling with t = 10^-5, negative sampling with 10 samples. We build vectors only for tokens observed 20 times or more in the corpus.

We then compute the cosine similarity of the vector
representation for a MWE candidate with the vectors of its
constituent words, and take the arithmetic mean. 
In scoring
the compositionality of a candidate, we do not measure the
cosine similarity of the MWE with any stop words it may
contain, as stop words may be assumed to be semantically
uninformative.
* Stop words are taken here to be the 50 most frequent words in the vocabulary.

In [12]:
# Stopwords from corpus - 50 most frequent

with open(datapath+'/Corpora/wiki/simple_20200601/Tagged/stop.pkl', 'rb') as pfile:
    stop = pickle.load(pfile)

In [13]:
stop

{'0|CD',
 '1|CD',
 '2|CD',
 '3|CD',
 '4|CD',
 'A|DT',
 'He|PRP',
 'In|IN',
 'It|PRP',
 'The|DT',
 'also|RB',
 'and|CC',
 'an|DT',
 'are|VBP',
 'as|IN',
 'at|IN',
 'a|DT',
 'be|VB',
 'born|VBN',
 'by|IN',
 'can|MD',
 'first|JJ',
 'for|IN',
 'from|IN',
 'had|VBD',
 'has|VBZ',
 'he|PRP',
 'his|PRP$',
 'in|IN',
 'is|VBZ',
 'it|PRP',
 'not|RB',
 'of|IN',
 'one|CD',
 'on|IN',
 'or|CC',
 'people|NNS',
 's|POS',
 'that|IN',
 'that|WDT',
 'they|PRP',
 'the|DT',
 'to|IN',
 'to|TO',
 'was|VBD',
 'were|VBD',
 'which|WDT',
 'with|IN',
 '|HYPH',
 '–|SYM'}

In [14]:
ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/simple_20200601/Tagged/ngram_eval.pkl')

ngram_eval

,ngram,freq,poisson,len,batch
0,"(Grădina|NNP, Zoologică|NNP)",20,-606.668931,2,1
1,"(Karlovy|NNP, Vary|NNP)",20,-606.668931,2,1
2,"(Ving|NNP, Rhames|NNP)",20,-606.668931,2,1
3,"(waystations|NNS, shuku|NN)",20,-608.076717,2,1
4,"(Aer|NNP, Lingus|NNP)",20,-609.419001,2,1
...,...,...,...,...,...
149995,"(Cena|NNP, and|CC)",56,-2732.579707,2,5
149996,"(and|CC, keeping|VBG)",56,-2732.579707,2,1
149997,"(province|NN, has|VBZ, a|DT)",40,-2732.586816,3,6
149998,"(speak|VBP, a|DT)",56,-2732.604570,2,2


In [15]:
min_freq = 20
eval_count = 150000

batch_count = max(ngram_eval.batch)

In [16]:
ngram_eval.batch.value_counts()

 1     47772
 2     33213
 3     25419
 4     15895
 5     10517
 6      6533
 7      3782
 8      2398
 9      1434
 10     1073
-2       732
 11      317
-1       282
 14      239
 12      227
 13       96
 15       71
Name: batch, dtype: int64

In [17]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [18]:
def mwe_score(exp, model, stats_frame):
    
    # Combined token for MWE
    mwetoken = '+'.join(exp)

    # Stopwords - 1 if component is a stopword, 0 if present, -1 if simplex word missing from vocab, -2 if MWE missing
    sws = []
    # Component vectors
    cvs = []

    #  Neighbours in original & MWE-aware space
    #oldn = []
    #newn = []

    # List of individual word similarities (where present in the vocab)
    css = []

    # Empty array
    earr = np.empty(400)
    earr[:] = np.nan

    # Check that combined token exists in the vocab. This protects against inflation of n-gram counts caused by repeats
    #  of the same token (e.g. in lists like https://simple.wikipedia.org/wiki/List_of_cities,_towns_and_villages_in_Fars_Province)
    if mwetoken in batch_model.wv.vocab:

        mwv = model.wv[mwetoken]

        for w in exp:
            if w in model.wv.vocab:
                cvs.append(model.wv[w])

                #oldn.append(batch_model.wv.most_similar(w, topn=5))

                if w in stop:
                    sws.append(1)
                    css.append(np.nan)
                else:
                    sws.append(0)
                    css.append(cosim(model.wv[w], mwv ))

            # If component is absent from vocab
            else:
                sws.append(-1)
                cvs.append(earr)
                css.append(np.nan)

                #oldn.append([])

        #  Mean cosim
        if min(sws) >= 0:
            cs = np.nanmean(css)
        else:
            cs = np.nan

        #newn = batch_model.wv.most_similar(mwetoken, topn=5)

    # Combined token missing from vocab - mark with defaults
    else:
        sws = [-2]
        mwv = np.empty(400)
        mwv[:] = np.nan
        cs = np.nan


    # Append to stats df
    return stats_frame.append({
        'ngram'  : exp,
        'stopwords' : sws,
        'mwe_vector' : mwv,
        'component_vectors' : cvs,
        'component_cosims'  : css,
        'cosine_sim'  : cs,
        #'base_nearest': oldn,
        #'mwe_nearest' : newn,
    }, ignore_index=True)

In [19]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    print('Processing batch {} of {}'.format(bb+1,batch_count))
    
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Load model
    print('Loading w2v model')

    batch_model = Word2Vec.load(datapath+'/Models/1 w2v/Tagged/simp_tagged_batch{}.model'.format(bb+1))
    
    print('Gathering MWE stats')
    
    # For each MWE, evaluate stats. Record vectors (in case we want to calculate different metrics later).
    statsf = pd.DataFrame(columns=['ngram', 'stopwords', 'mwe_vector', 'component_vectors', 'component_cosims', 
                                   'cosine_sim']) # , 'base_nearest', 'mwe_nearest'
    batch_len = len(batch_dfs[bb].ngram)
    if batch_len >= 20000: 
        printer = 5000
    elif batch_len >= 5000: 
        printer = 1000
    else:
        printer = 200
        
    _i = 0
    
    for exp in batch_dfs[bb].ngram:
        if _i % printer == 0:
            print(' MWE '+str(_i)+'/'+str(batch_len)+': '+'+'.join(exp))
        _i += 1
        
        statsf = mwe_score(exp, batch_model, statsf)
        
    #  Join back onto DataFrame
    batch_dfs[bb] = batch_dfs[bb].merge(statsf, on='ngram')
    

Processing batch 1 of 15
Loading w2v model
Gathering MWE stats
 MWE 0/47772: Grădina|NNP+Zoologică|NNP
 MWE 5000/47772: gun|NN+emplacements|NNS
 MWE 10000/47772: Moose|NNP+Jaw|NNP
 MWE 15000/47772: Dutch|NNP+War|NNP
 MWE 20000/47772: July|NNP+1962|CD
 MWE 25000/47772: to|TO+organise|VB
 MWE 30000/47772: different|JJ+depending|VBG
 MWE 35000/47772: Nobel|NNP+Memorial|NNP+Prize|NNP
 MWE 40000/47772: for|IN+digital|JJ
 MWE 45000/47772: the|DT+truck|NN
Processing batch 2 of 15
Loading w2v model
Gathering MWE stats
 MWE 0/33213: Senateacting|NNP+asGovernor|NNP
 MWE 5000/33213: airplane|NN+at|IN
 MWE 10000/33213: in|IN+battles|NNS
 MWE 15000/33213: 20|CD+3
 MWE 20000/33213: His|PRP$+funeral|NN+was|VBD
 MWE 25000/33213: written|VBN+many|JJ+books|NNS
 MWE 30000/33213: Louis|NNP+XVI|NNP+of|IN
Processing batch 3 of 15
Loading w2v model
Gathering MWE stats
 MWE 0/25419: Ilhwa|NNP+Chunma|NNP
 MWE 5000/25419: 1844|CD+and|CC
 MWE 10000/25419: DNS|NN+Ret|NN+Ret|NN
 MWE 15000/25419: excellence|NN+in|I

In [20]:
# Merge dataframes, sort by compositionality metric, export

# Also want the default batches with batch no < 0
all_batches = ngram_eval[ngram_eval.batch < 0]

for d in range(batch_count):
    print('Appending batch {} of '.format(d+1)+str(batch_count))
    all_batches = all_batches.append(batch_dfs[d])
    
all_batches = all_batches.sort_values('cosine_sim')
all_batches = all_batches.reset_index(drop=True)

all_batches.to_csv(datapath+'/Models/1 w2v/Tagged/Results/simp_tagged_output_001.csv', index=False)

Appending batch 1 of 15
Appending batch 2 of 15
Appending batch 3 of 15
Appending batch 4 of 15
Appending batch 5 of 15
Appending batch 6 of 15
Appending batch 7 of 15
Appending batch 8 of 15
Appending batch 9 of 15
Appending batch 10 of 15
Appending batch 11 of 15
Appending batch 12 of 15
Appending batch 13 of 15
Appending batch 14 of 15
Appending batch 15 of 15


In [21]:
all_batches

,ngram,freq,poisson,len,batch,stopwords,mwe_vector,component_vectors,component_cosims,cosine_sim
0,"(was|VBD, born|VBN, here|RB)",32,-2119.509496,3,8,"[1, 1, 0]","[-0.030439487, -0.035150945, 0.04269958, 0.129...","[[1.4429288, 0.8308226, -0.5883029, 0.38957492...","[nan, nan, -0.21604778]",-0.216048
1,"(s|POS, then|RB, |HYPH)",20,-1419.198013,3,4,"[1, 0, 1]","[-0.11279172, -0.14122055, 0.17148244, 0.16153...","[[0.1266791, -0.46722043, 0.7148848, 0.8804943...","[nan, -0.15213177, nan]",-0.152132
2,"(this|DT, time|NN, for|IN)",20,-1389.097772,3,9,"[0, 0, 1]","[-0.13903588, -6.825852e-05, 0.09086386, 0.059...","[[1.0234635, 0.8374039, 0.8275207, 0.03407207,...","[-0.15749776, -0.13361101, nan]",-0.145554
3,"(other|JJ, two|CD, being|VBG)",28,-1831.298757,3,10,"[0, 0, 0]","[-0.07178444, -0.13937448, 0.059084922, -0.004...","[[0.9786141, -0.79361075, 0.7509054, -0.460476...","[-0.13134842, -0.09783523, -0.19934192]",-0.142842
4,"(In|IN, This|DT)",26,-1380.814735,2,7,"[1, 0]","[-0.023711605, -0.10690656, 0.09365819, 0.1435...","[[-0.14552516, 0.023125043, 0.5271751, -1.5952...","[nan, -0.13744499]",-0.137445
...,...,...,...,...,...,...,...,...,...,...
149995,"(%|NN, 7|CD, %|NN)",20,-1332.973488,3,13,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN
149996,"(the|DT, Were|VBN, |HYPH)",22,-1422.855539,3,14,"[1, -1, 1]","[-0.1140692, -0.08132922, 0.064981386, 0.32641...","[[0.6643532, 0.13470931, 1.1582221, -0.4677486...","[nan, nan, nan]",NaN
149997,"(of|IN, Schwarzburg|NNP, |HYPH)",24,-1551.293123,3,14,"[1, -1, 1]","[-0.11180132, -0.06236225, -0.029179525, 0.233...","[[1.138536, -0.0335655, 0.3372522, -0.79551035...","[nan, nan, nan]",NaN
149998,"(the|DT, Nanboku|NNP, |HYPH)",33,-2145.863245,3,14,"[1, -1, 1]","[0.15855353, -0.10456044, 0.23011334, 0.045107...","[[0.6643532, 0.13470931, 1.1582221, -0.4677486...","[nan, nan, nan]",NaN


In [ ]:
#type(all_batches.stopwords[0])

In [ ]:
# RuntimeWarning: Mean of empty slice
#  Appear to be taking the np.nanmean() of a list of NaNs

def inval(inlist,val=0):
    if type(inlist) != list:
        return False
    return val in inlist

#nozero = all_batches[~all_batches.stopwords.apply(inval)]

#minone = nozero[nozero.stopwords.apply(inval,val=-1)]

#minone[minone.stopwords.apply(inval,val=1)]

In [22]:
simp_res_light = all_batches.drop(columns=['mwe_vector', 'component_vectors'])

In [23]:
simp_res_light.to_csv(datapath+'/Models/1 w2v/Tagged/Results/simp_tagged_light_001.csv', index=False)